# QUBO formulation of polynomial equation

# Use case

To illustrate the metohod we are taking the equation of the two node water system that reads:

$$
-1 - x_0 + x_1 = 0 \\
 1 - x_1 = 0 \\
 2 - q x_0^2 - x_2 = 0 \\
 -p x_1^2 + x_2 - x_3 = 0 
$$

with $q$ and $p$ paramerers taking disrete values either 1 or 2

In [1]:
import numpy as np

def nlfunc(input):
    x0,x1,x2,x3 = input
    q, p = parameters

    def f0():
        return -1 - x0 + x1
    
    def f1():
        return 1 - x1
    
    def f2():
        return 2  - q*x0**2  - x2

    def f3():
        return -p*x1**2 + x2 - x3
    
    return np.array([f0(), f1(), f2(), f3()])



## Classical Solution

The solution of such a small system can be obtained by newton raphson

In [2]:
from quantum_newton_raphson.newton_raphson import newton_raphson

parameters_list = [[i, j] for i in range(4) for j in range(4)] 

for parameters in parameters_list:
    initial_point = np.random.rand(4)
    res = newton_raphson(nlfunc, initial_point)
    assert np.allclose(nlfunc(res.solution), 0)
    print(parameters, res.solution)

[0, 0] [1.97361016e-11 1.00000000e+00 2.00000000e+00 2.00000000e+00]
[0, 1] [2.3854661e-21 1.0000000e+00 2.0000000e+00 1.0000000e+00]
[0, 2] [ 2.77586915e-17  1.00000000e+00  2.00000000e+00 -2.38723485e-11]
[0, 3] [-4.6259067e-17  1.0000000e+00  2.0000000e+00 -1.0000000e+00]
[1, 0] [8.32667906e-17 1.00000000e+00 2.00000000e+00 2.00000000e+00]
[1, 1] [1.11021173e-16 1.00000000e+00 2.00000000e+00 1.00000000e+00]
[1, 2] [-5.55113153e-17  1.00000000e+00  2.00000000e+00  2.11775042e-13]
[1, 3] [ 1.62400434e-22  1.00000000e+00  2.00000000e+00 -1.00000000e+00]
[2, 0] [-4.69045012e-21  1.00000000e+00  2.00000000e+00  2.00000000e+00]
[2, 1] [-1.01094061e-21  1.00000000e+00  2.00000000e+00  1.00000000e+00]
[2, 2] [5.55097881e-17 1.00000000e+00 2.00000000e+00 0.00000000e+00]
[2, 3] [-1.0004285e-21  1.0000000e+00  2.0000000e+00 -1.0000000e+00]
[3, 0] [-1.11021835e-16  1.00000000e+00  2.00000000e+00  2.00000000e+00]
[3, 1] [-8.32667269e-17  1.00000000e+00  2.00000000e+00  1.00000000e+00]
[3, 2] [ 5

/home/nico/QuantumApplicationLab/QuantumNewtonRaphson/quantum_newton_raphson/utils.py:74: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn("spsolve requires A be CSC or CSR matrix format", SparseEfficiencyWarning)


## 2. QUBO formalism for linear systems

The Quandratic Unconstrainted Binary Optimization problem, or QUBO, allows to minimize the cost function :

$$
E(x) = x^{T}Qx
$$

where the variables $x_i$ are binaries, i.e. the are 0 or 1. The equation above can be rewritten as :

$$
E(x) = \sum_i Q_{ii}x_i + \sum_{ij} Q_{ij}x_ix_j
$$

that is very similar to the Ising model, basis of the quantum annealler architecture. 

### Encoding real numbers in binary variables

In the QUBO problems, variables are binaries and we of course want to solve for real numbers in our case. There ar e different ways to encode real numbers in multiple binaries. In our case since the variables are between -1.0 and 1.0 we can use the following encoding : 

$$
r_i = a \sum_n x_n 2^{n} - x_{k+n} 2^{n} 
$$

where $a$ is a normalization constant. THis encoding is created in the `SolutionVector` class that allows to encode/decode real numbers in a series of binaries variables. We use here the `RealUnitQbitEncoding` to obtain real numbers between -1 and 1. The number of qbit controls the precision of the reals we can obtain.

### Polynomial equation

We first write the polynomial equation as follow (https://www.nature.com/articles/s41598-019-46729-0) 

$$
F(X) = 0
$$

with

$$
F_i = P_i^{(0)} + \sum_j P_{ij}^{(1)}x_j + \sum_{jk} P_{ijk}^{(2)}x_j x_k  = 0
$$

To solve the system we minimize the residual sum of square

$$
\chi^2 = [P^{(0)} + P^{(1)} X + P^{(2)} X X.T ]^2
$$


# Use case

To illustrate the metohod we are taking the equation of the two node water system that reads:

$$
-1 - x_0 + x_1 = 0 \\
 1 - x_1 = 0 \\
 2 - q x_0^2 - x_2 = 0 \\
 -p x_1^2 + x_2 - x_3 = 0 \\
 c(q + p) \rightarrow 0 
$$

with $q$ and $p$ paramerers taking disrete values either 1 or 2

In [3]:
import numpy as np
import sparse
def define_matrices():
    
    # system of equations
    num_equations = 5
    num_variables = 6

    P0 = np.zeros((num_equations,1))
    P0[0] = -1
    P0[1] = 1
    P0[2] = 2
    P0[3] = 0

    P1 = np.zeros((num_equations, num_variables))
    P1[0, 0] = -1
    P1[0, 1] =  1

    P1[1, 1] = -1

    P1[2, 2] = -1

    P1[3, 2] =  1 
    P1[3, 3] = -1

    # cost
    c = 1E-1
    P1[4,4] = c
    P1[4,5] = c
   

    P2 = np.zeros((num_equations, num_variables, num_variables))


    P3 = np.zeros((num_equations, num_variables, num_variables, num_variables))
    P3[2, 0, 0, 4] = -1
    P3[3, 1, 1, 5] = -1


    return sparse.COO(P0), sparse.COO(P1), sparse.COO(P2), sparse.COO(P3)

matrices = define_matrices()

## 3. Solving the system

We will use here the `SimulatedAnnealingSampler` to be able to run that code locally. Quantum solvers are available through the Leap cloud service.

In [4]:
from qubops.qubo_poly import QUBO_POLY
from qubops.encodings import PositiveQbitEncoding
import dimod
# options = {'num_reads':20, 'num_qbits':2, 'sampler':dimod.ExactSolver(), 
#            'encoding': PositiveQbitEncoding}
# qubops=QUBO_POLY(options)
# sol = qubops.solve(matrices)
# sol

# Switch sampler

In [5]:
import neal
sampler = neal.SimulatedAnnealingSampler()
options = {'num_reads':100, 'num_qbits':2, 'sampler':sampler, 
           'encoding': PositiveQbitEncoding}
qubops=QUBO_POLY(options)

sol = qubops.solve(matrices)
sol

array([0., 1., 2., 2., 0., 0.])

# Add slack variables

In [19]:
bqm = qubops.create_qubo_matrix(qubops.x)
# slacks1 = bqm.add_linear_inequality_constraint([("x_003_001",1), ("x_003_002",2)], lagrange_multiplier=2/3, label="head1", lb=0, ub=1)
slacks2 = bqm.add_linear_inequality_constraint(qubops.all_expr[3], lagrange_multiplier=1, label="head2", lb=0, ub=1)

In [20]:
import neal
sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads = 100)

In [21]:
sol = sampleset.lowest()
idx, vars, data = qubops.extract_data(sol)
sol = qubops.solution_vector.decode_solution(data)
sol

array([0., 1., 2., 1., 0., 1.])

In [9]:
parameters = sol[4:]
nlfunc(sol[:4])

array([0., 0., 0., 0.])

In [16]:
qubops.all_vars

['x_001_001',
 'x_001_002',
 'x_002_001',
 'x_002_002',
 'x_003_001',
 'x_003_002',
 'x_004_001',
 'x_004_002',
 'x_005_001',
 'x_005_002',
 'x_006_001',
 'x_006_002']

In [17]:
qubops.all_expr

[[('x_001_001', 1), ('x_001_002', 2)],
 [('x_002_001', 1), ('x_002_002', 2)],
 [('x_003_001', 1), ('x_003_002', 2)],
 [('x_004_001', 1), ('x_004_002', 2)],
 [('x_005_001', 1), ('x_005_002', 2)],
 [('x_006_001', 1), ('x_006_002', 2)]]